In [ ]:
import polars as pl
import polars_ds as pds
import numpy as np

In [ ]:
df = pl.DataFrame(
    {
        "id": range(5),
        "conn": [[1, 2, 3, 4], [2, 3], [4], [0, 1, 2], [1]],
        "cost": [[0.4, 0.3, 0.2, 0.1], [0.1, 1], [0.5], [0.1, 0.1, 0.1], [0.1]],
    }
).with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("conn").list.eval(pl.element().cast(pl.UInt32))
)

df = df.explode(pl.col("conn"), pl.col("cost"))
df

In [ ]:
df.select(
    pl.col("id")
    .graph.shortest_path(
        link = pl.col("conn"),
        target=1,
        cost=pl.col("cost"),
    ).alias("path")
).unnest("path")

In [ ]:
df = pds.random_data(15_000, n_cols=4)
df.head()

In [ ]:
df = df.select(
    pl.col("feature_1").alias("B_1"),
    pl.col("feature_2").alias("B_2"),
    pl.col("feature_3").alias("B_3"),
    pl.col("feature_4").alias("A"),
)
df.head()

In [ ]:
df.select(
    pds.query_lstsq(pl.col("B_1"), pl.col("B_2"), pl.col("B_3"), target = pl.col("A"), return_pred=True, add_bias=True)
)

In [ ]:
df.with_columns(
    pl.col('A').num.lstsq(*[pl.col(c) for c in df.columns if c.startswith("B_")], return_pred=True).struct.field('resid')
)

In [ ]:
# pl.col('A').num.lstsq(pl.col('^B_.*$'), return_pred=True).struct.field('resid')

In [ ]:
print(df.columns)
df.with_columns(
    pl.col('A').num.lstsq(pl.col('^B_.*$'), return_pred=True) # .struct.field('resid')
).head()

In [ ]:
print(
    
df.select(
    pl.col("row_num"),
    pds.query_radius_ptwise(
        pl.col("feature_1"), pl.col("feature_2"), pl.col("feature_3"), # Columns used as the coordinates in n-d space
        index = pl.col("row_num"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

In [ ]:
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True

In [ ]:
import scipy

In [ ]:
df.select(pl.col("feature_1").num.rfft())

In [ ]:
df.select(pl.col("feature_1").num.rfft2())

In [ ]:
import polars as pl
import polars_ds as pld  # noqa
from pandas._testing import makeTimeDataFrame

In [ ]:
df = makeTimeDataFrame()
df = df.rename(columns={'B': 'B_1', 'C': 'B_2', 'D': 'B_3', })
df = pl.from_pandas(df, include_index=True)